# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)

# İş Problemi (Business Problem)

FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu davranış öbeklenmelerine göre gruplar oluşturulacak.


## Veri Seti Hikayesi

Veri seti son alışverişlerini 2020 - 2021 yıllarında OmniChannel(hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

**Değişkenler**
* master_id: Eşsiz müşteri numarası
* order_channel : Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile, Offline)
* last_order_channel : En son alışverişin yapıldığı kanal
* first_order_date : Müşterinin yaptığı ilk alışveriş tarihi
* last_order_date : Müşterinin yaptığı son alışveriş tarihi
 * last_order_date_online : Muşterinin online platformda yaptığı son alışveriş tarihi
 * last_order_date_offline : Muşterinin offline platformda yaptığı son alışveriş tarihi
 * order_num_total_ever_online : Müşterinin online platformda yaptığı toplam alışveriş sayısı
 * order_num_total_ever_offline : Müşterinin offline'da yaptığı toplam alışveriş sayısı
 * customer_value_total_ever_offline : Müşterinin offline alışverişlerinde ödediği toplam ücret
 * customer_value_total_ever_online : Müşterinin online alışverişlerinde ödediği toplam ücret
 * interested_in_categories_12 : Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

In [1]:
import numpy as np
import pandas as pd
import datetime as dt

### GÖREV 1: Veriyi Anlama (Data Understanding) ve Hazırlama

In [2]:
df_ = pd.read_csv('flo_data_20k.csv')
df = df_.copy()

In [3]:
# a. İlk 10 gözlem,
df.head(10)


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.0,2.0,150.87,49.99,[KADIN]
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.0,1.0,59.99,315.94,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.0,1.0,49.99,113.64,[COCUK]
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.0,2.0,120.48,934.21,"[ERKEK, COCUK, KADIN]"
9,1143f032-440d-11ea-8b43-000d3a38a36f,Mobile,Mobile,2019-07-30,2020-10-04,2020-10-04,2019-07-30,1.0,1.0,69.98,95.98,"[KADIN, AKTIFSPOR]"


In [4]:
# b. Değişken isimleri,
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

In [5]:
# c. Betimsel istatistik,
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0,3.110855,4.225647,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.0,1.913913,2.062880,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.0,253.922597,301.532853,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.0,497.321690,832.601886,12.99,149.98,286.46,578.44,45220.13


In [6]:
# d. Boş değer,
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [7]:
# e. Değişken tipleri, incelemesi yapınız.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [9]:
# 3. Omnichannel müşterilerin hem online'dan hemde offline platformlardan alışveriş yaptığını ifade etmektedir. Herbir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturun.

df['toplam_alisveris_sayisi'] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["toplam_harcama"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,toplam_alisveris_sayisi,toplam_harcama
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98


In [11]:
convert =["first_order_date","last_order_date","last_order_date_online","last_order_date_offline"]
df[convert] = df[convert].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [12]:
# 5. Alışveriş kanallarındaki müşteri sayısının, ortalama alınan ürün sayısının ve ortalama harcamaların dağılımına bakınız.

df.groupby('order_channel').agg({'toplam_alisveris_sayisi':'mean',
                                'toplam_harcama':'mean',
                                 })

,toplam_alisveris_sayisi,toplam_harcama
order_channel,,
Android App,5.504897,823.492655
Desktop,3.992687,588.782984
Ios App,5.418637,891.634285
Mobile,4.440598,620.275125


In [13]:
# 6. En fazla kazancı getiren ilk 10 müşteriyi sıralayınız.

df.groupby('master_id').agg({'toplam_harcama': 'sum'}).sort_values(by='toplam_harcama', ascending=False).head(10)

,toplam_harcama
master_id,
5d1c466a-9cfd-11e9-9897-000d3a38a36f,45905.10
d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,36818.29
73fd19aa-9e37-11e9-9897-000d3a38a36f,33918.10
7137a5c0-7aad-11ea-8f20-000d3a38a36f,31227.41
47a642fe-975b-11eb-8c2a-000d3a38a36f,20706.34
a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,18443.57
d696c654-2633-11ea-8e1c-000d3a38a36f,16918.57
fef57ffa-aae6-11e9-a2fc-000d3a38a36f,12726.10
cba59206-9dd1-11e9-9897-000d3a38a36f,12282.24


In [14]:
# 7. En fazla siparişi veren ilk 10 müşteriyi sıralayınız.
df.groupby('master_id').agg({'toplam_alisveris_sayisi': 'sum'}).sort_values(by='toplam_alisveris_sayisi', ascending=False).head(10)

,toplam_alisveris_sayisi
master_id,
5d1c466a-9cfd-11e9-9897-000d3a38a36f,202.0
cba59206-9dd1-11e9-9897-000d3a38a36f,131.0
a57f4302-b1a8-11e9-89fa-000d3a38a36f,111.0
fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,88.0
329968c6-a0e2-11e9-a2fc-000d3a38a36f,83.0
73fd19aa-9e37-11e9-9897-000d3a38a36f,82.0
44d032ee-a0d4-11e9-a2fc-000d3a38a36f,77.0
b27e241a-a901-11e9-a2fc-000d3a38a36f,75.0
d696c654-2633-11ea-8e1c-000d3a38a36f,70.0


In [15]:
# 8. Veri ön hazırlık sürecini fonksiyonlaştırınız.

def data_preprocessing(dataframe):
    dataframe['toplam_alisveris_sayisi'] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["toplam_harcama"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    convert = ["first_order_date", "last_order_date", "last_order_date_online", "last_order_date_offline"]
    dataframe[convert] = dataframe[convert].apply(pd.to_datetime)

    dataframe.groupby('order_channel').agg({'toplam_alisveris_sayisi':'mean',
                                    'toplam_harcama':'mean',
                                     })

    dataframe.groupby('master_id').agg({'toplam_harcama': 'sum'}).sort_values(by='toplam_harcama', ascending=False).head(10)
    dataframe.groupby('master_id').agg({'toplam_alisveris_sayisi': 'sum'}).sort_values(by='toplam_alisveris_sayisi',ascending=False).head(10)


data_preprocessing(df)

df.head()


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,toplam_alisveris_sayisi,toplam_harcama
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98


In [18]:
# GÖREV 2: RFM Metriklerinin Hesaplanması

# Recency, Frequency, Monetary
df.head()
df["last_order_date"].max() # 2021-05-30

Timestamp('2021-05-30 00:00:00')

In [19]:
today_date = dt.datetime(2021, 7, 1)
type(today_date)


datetime.datetime

In [22]:
rfm = df.groupby("master_id").agg({
    "last_order_date": lambda date: (today_date -date.max()).days,
    "toplam_alisveris_sayisi": lambda alisveris: alisveris.sum(),
    "toplam_harcama": lambda TotalPrice: TotalPrice.sum()
})
rfm.columns = ["recency", "frequency", "monetary"]

rfm.head(10)

,recency,frequency,monetary
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,40,5.0,776.07
00034aaa-a838-11e9-a2fc-000d3a38a36f,328,3.0,269.47
000be838-85df-11ea-a90b-000d3a38a36f,243,4.0,722.69
000c1fe2-a8b7-11ea-8479-000d3a38a36f,57,7.0,874.16
000f5e3e-9dde-11ea-80cd-000d3a38a36f,50,7.0,1620.33
00136ce2-a562-11e9-a2fc-000d3a38a36f,233,2.0,359.45
00142f9a-7af6-11eb-8460-000d3a38a36f,55,3.0,404.94
0014778a-5b11-11ea-9a2c-000d3a38a36f,56,3.0,727.43
0018c6aa-ab6c-11e9-a2fc-000d3a38a36f,156,2.0,317.91


In [23]:
# GÖREV 3: RF ve RFM Skorlarının Hesaplanması
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

In [24]:
rfm["RFM_SCORE"] = (rfm["recency_score"].astype(str) +
                    rfm["frequency_score"].astype(str))

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
master_id,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,40,5.0,776.07,5,4,4,54
00034aaa-a838-11e9-a2fc-000d3a38a36f,328,3.0,269.47,1,2,1,12
000be838-85df-11ea-a90b-000d3a38a36f,243,4.0,722.69,2,3,4,23
000c1fe2-a8b7-11ea-8479-000d3a38a36f,57,7.0,874.16,5,4,4,54
000f5e3e-9dde-11ea-80cd-000d3a38a36f,50,7.0,1620.33,5,4,5,54


In [25]:
# GÖREV 4: RF Skorlarının Segment Olarak Tanımlanması

seg_map = {
    r"[1-2][1-2]" : "hibernating",
    r"[1-2][3-4]" : "at_risk",
    r"[1-2]5": "cant_loose",
    r"3[1-2]": "about_to_sleep",
    r"33": "need_attention",
    r"[3-4][4-5]": "loyal_customers",
    r"41": "promising",
    r"51": "new_customers",
    r"[4-5][2-3]": "potential_loyalists",
    r"5[4-5]": "champions",
}

rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex=True) # birleştirilen skorlar seg_map i

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,40,5.0,776.07,5,4,4,54,champions
00034aaa-a838-11e9-a2fc-000d3a38a36f,328,3.0,269.47,1,2,1,12,hibernating
000be838-85df-11ea-a90b-000d3a38a36f,243,4.0,722.69,2,3,4,23,at_risk
000c1fe2-a8b7-11ea-8479-000d3a38a36f,57,7.0,874.16,5,4,4,54,champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,50,7.0,1620.33,5,4,5,54,champions


In [26]:
# GÖREV 5: Aksiyon zamanı!
# 1. Segmentlerin recency, frequnecy ve monetary ortalamalarını inceleyiniz.

rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])


recency        frequency           monetary      
                           mean count       mean count         mean count
segment                                                                  
about_to_sleep       144.031649  1643   2.406573  1643   361.649373  1643
at_risk              272.328997  3152   4.470178  3152   648.325038  3152
cant_loose           265.159129  1194  10.716918  1194  1481.652446  1194
champions             47.142187  1920   8.965104  1920  1410.708938  1920
hibernating          277.426303  3589   2.391474  3589   362.583299  3589
loyal_customers      112.557926  3375   8.356444  3375  1216.257224  3375
need_attention       143.037221   806   3.739454   806   553.436638   806
new_customers         47.976226   673   2.000000   673   344.049495   673
potential_loyalists   66.869744  2925   3.310769  2925   533.741344  2925
promising             88.694611   668   2.000000   668   334.153338   668

In [27]:
# 2. RFM analizi yardımı ile 2 case için ilgili profildeki müşterileri bulun ve müşteri id'lerini csv ye kaydediniz.

# a. FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri tercihlerinin üstünde. Bu nedenle markanın tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak iletişime geçilmek isteniliyor. Sadık müşterilerinden(champions,loyal_customers),ortalama 250 TL üzeri ve kadın kategorisinden alışveriş yapan kişiler özel olarak iletişim kuralacak müşteriler. Bu müşterilerin id numaralarını csv dosyasına yeni_marka_hedef_müşteri_id.cvs olarak kaydediniz.

rfm.head()
type(rfm)
pd.merge(df, rfm, on='master_id').head()
merge_df = pd.merge(df, rfm, on='master_id', how='left')
merge_df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,...,toplam_alisveris_sayisi,toplam_harcama,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,...,5.0,939.37,125,5.0,939.37,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,...,21.0,2013.55,135,21.0,2013.55,3,5,5,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,...,5.0,585.32,216,5.0,585.32,2,4,3,24,at_risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,...,2.0,121.97,165,2.0,121.97,3,1,1,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,...,2.0,209.98,116,2.0,209.98,3,1,1,31,about_to_sleep


In [29]:
merge_df["ortalama_harcama"] = merge_df["toplam_harcama"] / merge_df['toplam_alisveris_sayisi']
yeni_marka_hedef_musteri = merge_df[
    (merge_df["segment"].isin(["champions", "loyal_customers"])) &
    (merge_df["ortalama_harcama"] > 250) &
    (merge_df["interested_in_categories_12"] == "[KADIN]")
]
yeni_marka_hedef_musteri[["master_id"]].reset_index(drop=True).to_csv("yeni_marka_hedef_musteri.csv")

In [32]:
yeni_marka_hedef_musteri["master_id"]

3662     63caf5e4-e3be-11e9-ad00-000d3a38a36f
5457     78566b7e-174e-11ea-9213-000d3a38a36f
7272     d186774a-b095-11e9-9757-000d3a38a36f
8810     1348781e-134b-11ea-a001-000d3a38a36f
12778    95706470-1e38-11eb-8d74-000d3a38a36f
19346    e42a05aa-29e0-11ea-87cc-000d3a38a36f
Name: master_id, dtype: object

In [34]:
# b. Erkek ve Çoçuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte iyi müşteri olan ama uzun süredir alışveriş yapmayan kaybedilmemesi gereken müşteriler, uykuda olanlar ve yeni gelen müşteriler özel olarak hedef alınmak isteniliyor. Uygun profildeki müşterilerin id'lerini csv dosyasına indirim_hedef_müşteri_ids.csv olarak kaydediniz.

merge_df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,...,toplam_harcama,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment,ortalama_harcama
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,...,939.37,125,5.0,939.37,3,4,4,34,loyal_customers,187.874000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,...,2013.55,135,21.0,2013.55,3,5,5,35,loyal_customers,95.883333
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,...,585.32,216,5.0,585.32,2,4,3,24,at_risk,117.064000
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,...,121.97,165,2.0,121.97,3,1,1,31,about_to_sleep,60.985000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,...,209.98,116,2.0,209.98,3,1,1,31,about_to_sleep,104.990000


In [35]:
indirim_hedef_müşteri = merge_df[
    (merge_df["interested_in_categories_12"] == "[ERKEK, COCUK]") &
    (merge_df["segment"].isin(["cant_loose", "potential_loyalists", "about_to_sleep", "need_attention", "promising", "new_customers"]))
]

new_indirim_hedef_musteri = indirim_hedef_müşteri[["master_id"]].reset_index(drop=True)

new_indirim_hedef_musteri.to_csv("indirim_hedef_müşteri_ids.csv")

In [36]:
new_indirim_hedef_musteri

,master_id
0,6ece7c1c-4594-11ea-acaf-000d3a38a36f
1,9a981e7e-ee10-11e9-9346-000d3a38a36f
2,fdb42464-76bc-11ea-b23f-000d3a38a36f
3,06548210-bd28-11ea-aa61-000d3a38a36f
4,bc2f61c0-afde-11e9-9757-000d3a38a36f
...,...
71,7e6d1820-1fdf-11ea-87bf-000d3a38a36f
72,98928afc-2465-11ea-a30f-000d3a38a36f
73,5b010dc6-9de7-11e9-9897-000d3a38a36f
74,4343f726-391d-11ea-85d6-000d3a38a36f
